In [47]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import os
import pytz
from google.cloud import bigquery
from datetime import timedelta, datetime

In [48]:

timezone = pytz.timezone('US/Eastern')
current_datetime = datetime.now(tz=timezone)

In [49]:
table_name = current_datetime.strftime('%Y%m%d')

In [50]:
table_name

'20190826'

In [51]:
current_datetime.strftime('%Y%m%d %H:%M')

'20190826 16:53'

In [52]:
headers = {
    # 'Host': 'i.meizitu.net',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    #'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
    'Connection': 'keep-alive',
    'Accept-Encoding':'gzip, deflate, br'#,
#     'Upgrade-Insecure-Requests':'1'
}

In [53]:
s = requests.session()

In [54]:
url_re = s.get('https://www.msn.com', headers=headers)

In [55]:
url_re.status_code

200

In [56]:
df = []

In [57]:
soup = BeautifulSoup(url_re.text)

In [58]:
soup.select('img[alt="Business Insider Logo"]')

[<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=png"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif"/>,
 <img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=png"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif"/>,
 <img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=png"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif"/>]

In [59]:
#### main river ####
for item in soup.select('.todaystripe.grid')[0].select('ul')[0].findAll("li", recursive=False):
    if (item.select('img[alt="Business Insider Logo"]')) or (item.select('img[alt="INSIDER Logo"]')):
#     if (item.select('img[alt="The Daily Meal Logo"]')):
        river_position = 1
        river_name = 'top river'       
        if item['class'][0] == 'infopane-placeholder':
            item_block_position = 1
            item_block_type = 'swipenav'
            for item_block_link in item.find(class_='swipenav').findAll("li", recursive=False):
                if (item_block_link.select('img[alt="Business Insider Logo"]')) or (item_block_link.select('img[alt="INSIDER Logo"]')):
                    item_block_link_position = re.findall("\"o\":(\d+)",item_block_link['data-m'])[0]
#                     print(item_block_link_position)
#                     TEMP2 = item_block_link
                    article_title = item_block_link.select('span[aria-label]')[0]['aria-label']
                    article_link = item_block_link.a['href']
                    article_img = re.findall(r'\/\/(.*)\?',item_block_link.select('img')[0]['data-src'])[0]
                    article_img_small = re.findall('\/\/(.+)\"', item_block_link.a.img['data-src'])[0]
                    df.append([river_position, 
                                   river_name,
                                   item_block_position,
                                   item_block_type,
                                   item_block_link_position,
                                   article_title,
                                   article_link,
                                   article_img,
                                   article_img_small])
            
                    print(article_title, '\n', article_link, '\n',article_img,'\n',article_img_small )
        else:
            temp3 = item
            item_block_position = re.findall("\"o\":(\d+)",item['data-m'])[0]
            item_block_type = ' '.join(item['class'])
            item_block_link_position = 1
            article_title = item.select('span[aria-label]')[0]['aria-label']
            article_link = item.a['href']
            article_img = re.findall(r'\/\/(.*)\?',item.select('img')[0]['data-src'])[0]
            article_img_small = re.findall('\/\/(.+)\"', item.a.img['data-src'])[0]
            df.append([river_position, 
                                   river_name,
                                   item_block_position,
                                   item_block_type,
                                   item_block_link_position,
                                   article_tile,
                                   article_link,
                                   article_img,
                                   article_img_small])
            

organizations that fight for women's rights-slideshow 
 /en-us/lifestyle/causes-gender-equality/21-organizations-that-are-fighting-for-womens-rights-in-the-us-and-abroad/ss-AAFQWLI?li=BBnb7Kz 
 static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAFRfBF.img 
 static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAFRfBF.img?h=333&w=311&m=6&q=60&u=t&o=t&l=f&f=jpg&x=1359&y=1048


In [60]:
df

[[1,
  'top river',
  1,
  'swipenav',
  '20',
  "organizations that fight for women's rights-slideshow",
  '/en-us/lifestyle/causes-gender-equality/21-organizations-that-are-fighting-for-womens-rights-in-the-us-and-abroad/ss-AAFQWLI?li=BBnb7Kz',
  'static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAFRfBF.img',
  'static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAFRfBF.img?h=333&w=311&m=6&q=60&u=t&o=t&l=f&f=jpg&x=1359&y=1048']]

In [61]:
for index, item in enumerate(soup.select('.stripecontainer[aria-label]'), start = 2):
    if (item.select('img[alt="Business Insider Logo"]')) or item.select('img[alt="INSIDER Logo"]'):
#         print(item)
#         print(index)
        river_position = index
        river_name = item['aria-label']
#         a = item

        for item_block in item.find_all('ul'):
            if (item_block.select('img[alt="Business Insider Logo"]')) or item_block.select('img[alt="INSIDER Logo"]'):
#                 ib = item_block
                item_block_type = item_block['data-aop']
                item_block_position = re.findall("\"o\":(\d+)",item_block['data-m'])[0]
#                 print(item_block)
                for item_block_link in item_block.select('li'):
                    if (item_block_link.select('img[alt="Business Insider Logo"]')) or item_block_link.select('img[alt="INSIDER Logo"]'):
#                         ibl = item_block_link
#                         x = item_block_link
#                         print(item_block_link)
#                         print(len(item_block_link))
                        item_block_link_position = re.findall("\"o\":(\d+)",item_block_link['data-m'])[0]
                        article_tile = item_block_link.select('h3[aria-label]')[0]['aria-label']
                        article_link = item_block_link.a['href']
                        article_img = re.findall(r'\/\/(.*)\?',item_block_link.select('img')[0]['data-src'])[0]
                        article_img_small = re.findall('\/\/(.+)\"', item_block_link.a.img['data-src'])[0]
#                         artile_img = re.findall(r'\/\/(.*)\?',item_block_link.select('img')[0]['data-src'])[0]
                        df.append([river_position, 
                                   river_name,
                                   item_block_position,
                                   item_block_type,
                                   item_block_link_position,
                                   article_tile,
                                   article_link,
                                   article_img,
                                   article_img_small])
        
        
#     else:
#         print('***')
df = pd.DataFrame(df, columns = ['river_position',
                           'river_name',
                           'item_block_position',
                           'item_block_type',
                           'item_block_link_position',
                           'article_title',
                           'article_link',
                           'article_img',
                           'article_img_small'])
df

,river_position,river_name,item_block_position,item_block_type,item_block_link_position,article_title,article_link,article_img,article_img_small
0,1,top river,1,swipenav,20,organizations that fight for women's rights-sl...,/en-us/lifestyle/causes-gender-equality/21-org...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
1,3,news,1,secondary,1,fallout from russia missile disaster suggests ...,/en-us/news/world/fallout-from-russias-mysteri...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
2,3,news,4,secondary,1,is trump quieting the fears of american doomsd...,/en-us/news/us/doomsday-preppers-are-thinning-...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
3,8,lifestyle,2,tertiary,2,the explanation behind 6 popular design elemen...,/en-us/lifestyle/did-you-know/the-explanation-...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
4,8,lifestyle,3,tertiary,3,trump helicopter left scorch marks on buckingh...,/en-us/lifestyle/royals/its-ruined-queen-eliza...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
5,10,health & fitness,4,secondary,1,a trainer debunks the 7 biggest fitness myths ...,/en-us/health/fitness/a-trainer-whos-worked-wi...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...


In [62]:
df['time'] = current_datetime.strftime('%Y%m%d %H:%M')

In [77]:
df.set_index(df.columns[-1], inplace = True)   #df_bq.set_index(df_bq.columns[-1], inplace = True)

In [78]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/wzhang/Documents/WorkFile/GoogleAPI/insideradrevenue-3e1aa74bcbfe.json"

In [79]:
df

,river_position,river_name,item_block_position,item_block_type,item_block_link_position,article_title,article_link,article_img,article_img_small
time,,,,,,,,,
20190826 14:06,1,top river,1,swipenav,15,disneyland's massive marvel expansion: what to...,/en-us/travel/tripideas/a-massive-marvel-theme...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
20190826 14:06,1,top river,1,swipenav,18,royal photographer shares favorite photos of t...,/en-us/lifestyle/royals/a-royal-photographer-s...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
20190826 14:06,1,top river,1,swipenav,19,what happens to your body when you cut out dai...,/en-us/health/nutrition/heres-what-happens-to-...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
20190826 14:06,1,top river,1,swipenav,21,women whose inventions were credited to men-sl...,/en-us/lifestyle/causes-gender-equality/14-wor...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
20190826 14:06,3,news,2,tertiary,2,is trump quieting the fears of american doomsd...,/en-us/news/us/doomsday-preppers-are-thinning-...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
20190826 14:06,8,lifestyle,2,tertiary,3,the explanation behind 6 popular design elemen...,/en-us/lifestyle/did-you-know/the-explanation-...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
20190826 14:06,8,lifestyle,3,tertiary,3,trump helicopter left scorch marks on buckingh...,/en-us/lifestyle/royals/its-ruined-queen-eliza...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
20190826 14:06,10,health & fitness,4,secondary,1,a trainer debunks the 7 biggest fitness myths ...,/en-us/health/fitness/a-trainer-whos-worked-wi...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...
20190826 14:06,11,food & drink,2,tertiary,1,this calzone is stuffed with 10 pounds of past...,/en-us/foodanddrink/video/this-calzone-is-stuf...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...


In [80]:
client = bigquery.Client()

In [81]:
df['river_position'] = (df['river_position']).astype(int)
df['item_block_position'] = (df['item_block_position']).astype(int)
df['item_block_link_position'] = (df['item_block_link_position']).astype(int)

In [82]:
table_ref = client.dataset("MSN", project='insideradrevenue').table(
            "MSN_HP_{}".format(table_name))

job_config = bigquery.LoadJobConfig()
job_config.write_disposition = 'WRITE_APPEND'
job_config.autodetect = True
job = client.load_table_from_dataframe(
        df,
        table_ref,
        location='US',
        job_config=job_config)


In [8]:
soup.select('img[alt="Business Insider Logo"]')

[<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=png"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif"/>,
 <img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=png"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif"/>]

In [80]:
soup.select('.todaystripe.grid')[0].select('li')[0]['class'][0]

'infopane-placeholder'

In [120]:
len(soup.select('.todaystripe.grid')[0].select('ul')[0].findAll("li", recursive=False))

16

In [178]:
#### main river ####
for item in soup.select('.todaystripe.grid')[0].select('ul')[0].findAll("li", recursive=False):
#     if (item.select('img[alt="Business Insider Logo"]')) or (item.select('img[alt="INSIDER Logo"]')):
    if (item.select('img[alt="The Daily Meal Logo"]')):
        river_position = 1
        river_name = 'top river'       
        if item['class'][0] == 'infopane-placeholder':
            item_block_position = 1
            item_block_type = 'swipenav'
            for item_block_link in item.find(class_='swipenav').findAll("li", recursive=False):
                if (item_block_link.select('img[alt="Business Insider Logo"]')) or (item_block_link.select('img[alt="INSIDER Logo"]')):
                    item_block_link_position = re.findall("\"o\":(\d+)",item_block_link['data-m'])[0]
                    print(item_block_link_position)
                    TEMP2 = item_block_link
                    article_title = item_block_link.select('span[aria-label]')[0]['aria-label']
                    article_link = item_block_link.a['href']
                    article_img = re.findall(r'\/\/(.*)\?',item_block_link.select('img')[0]['data-src'])[0]
                    article_img_small = re.findall('\/\/(.+)\"', item_block_link.a.img['data-src'])[0]
            
                    print(article_title, '\n', article_link, '\n',article_img,'\n',article_img_small )
        else:
            temp3 = item
            item_block_position = re.findall("\"o\":(\d+)",item['data-m'])[0]
            item_block_type = ' '.join(item['class'])
            item_block_link_position = 1
            article_title = item.select('span[aria-label]')[0]['aria-label']
            article_link = item.a['href']
            article_img = re.findall(r'\/\/(.*)\?',item.select('img')[0]['data-src'])[0]
            article_img_small = re.findall('\/\/(.+)\"', item.a.img['data-src'])[0]
            
            
            
#         temp = item
#         article_img = 
#         article_img_small = 
#         print(item['class'])
        
        
        
        
# 'river_position',
#                            'river_name',
#                            'item_block_position',
#                            'item_block_type',
#                            'item_block_link_position',
#                            'article_title',
#                            'article_link',
#                            'article_img',
#                            'article_img_small'   


#                         article_tile = item_block_link.select('h3[aria-label]')[0]['aria-label']
#                         article_link = item_block_link.a['href']
#                         article_img = re.findall(r'\/\/(.*)\?',item_block_link.select('img')[0]['data-src'])[0]
#                         article_img_small = re.findall('\/\/(.+)\"', item_block_link.a.img['data-src'])[0]

In [194]:
re.findall(r'\/\/(.*)\?',temp3.select('img')[0]['data-src'])[0]

'static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAG55P4.img'

In [191]:
re.findall("\"o\":(\d+)",temp3['data-m'])[0]

'2'

In [179]:
temp3

<li class="pos1 replacead23 smalla smalla-twoimages media photo" data-aop="river&gt;1" data-id="183" data-m='{"i":183,"p":117,"n":"HeadlineItemViewModel","y":13,"l":"AAG4Sxa","h":0,"v":"foodanddrink","c":"foodnews","o":2}'>
<a href="/en-us/foodanddrink/foodnews/20-things-you-didnt-know-about-french-fries/ss-AAG4Sxa?li=BBnbfcL">
<img alt='Of all the &lt;a href="https://www.thedailymeal.com/cook/50-great-ways-cook-pota...' data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAG55P4.img?h=250&amp;w=300&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=jpg"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif" title="french fries - istockphoto.com"/><img alt='Of all the &lt;a href="https://www.thedailymeal.com/cook/50-great-ways-cook-pota...' data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAG55P4.img?h=75&amp;w=100&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=jpg"}' src="//static-globa

In [189]:
'-'.join(temp3['class'])

'pos1-replacead23-smalla-smalla-twoimages-media-photo'

In [180]:
re.findall("\"o\":(\d+)",temp3['data-m'])[0]

'2'

In [161]:
TEMP2.select('span["aria-label"]')

{'tabindex': '0',
 'class': ['single', 'hasimage', 'media', 'photo'],
 'data-aop': 'slide19>single ',
 'data-id': '175',
 'data-m': '{"i":175,"p":118,"n":"single","y":8,"o":20}'}

In [167]:
TEMP2.select('span[aria-label]')[0]['aria-label']

"actors you've seen everywhere whose names you still don't know-slideshow"

TEMP2.article_link

In [169]:
TEMP2.a['href']

'/en-us/movies/gallery/20-actors-that-have-been-in-dozens-of-movies-and-tv-shows-whose-name-you-probably-still-dont-know/ss-AAGb2zQ?li=BBnb7Kz'

In [148]:
temp['class'][0]

'infopane-placeholder'

In [146]:
for xyz in temp.find(class_='swipenav').findAll("li", recursive=False):
    print(xyz['class'])
    print(xyz['data-m'])
    print(re.findall("\"o\":(\d+)",xyz['data-m'])[0])

['mosaic', 'hasimage', 'selected']
{"i":119,"p":118,"n":"mosaic","y":8,"o":1}
1
['mosaic']
{"i":123,"p":118,"n":"mosaic","y":8,"o":2,"f":"Y_cec865fe-ed84-49a8-b88a-4a23446ea7f1"}
2
['mosaic', 'hasimage']
{"i":127,"p":118,"n":"mosaic","y":8,"o":3}
3
['mosaic', 'hasimage']
{"i":131,"p":118,"n":"mosaic","y":8,"o":4}
4
['single', 'hasimage']
{"i":135,"p":118,"n":"single","y":8,"o":5}
5
['single', 'hasimage', 'media', 'photo']
{"i":137,"p":118,"n":"single","y":8,"o":6}
6
['head-to-head', 'hasimage']
{"i":139,"p":118,"n":"headtohead","y":8,"o":7}
7
['single', 'hasimage']
{"i":142,"p":118,"n":"single","y":8,"o":8}
8
['mosaic', 'hasimage']
{"i":144,"p":118,"n":"mosaic","y":8,"o":9}
9
['single', 'hasimage']
{"i":148,"p":118,"n":"single","y":8,"o":10}
10
['mosaic', 'hasimage']
{"i":150,"p":118,"n":"mosaic","y":8,"o":11}
11
['single', 'hasimage']
{"i":154,"p":118,"n":"single","y":8,"o":12}
12
['single', 'hasimage', 'media', 'photo']
{"i":156,"p":118,"n":"single","y":8,"o":13}
13
['mosaic', 'hasim

In [131]:
temp.select('img[alt="Business Insider Logo"]')[0].parent.parent

<div class="caption">
<span aria-label="what cities looked like before the epa regulated pollution-slideshow" class="title">What cities looked like before the EPA regulated pollution</span>
<span class="sourcename">
<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=png"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif"/>Business Insider        </span>
</div>

In [102]:
soup.select('.todaystripe.grid')[0].attrs

{'class': ['todaystripe',
  'grid',
  'layout-large',
  'popnow',
  'popnow3',
  'edpicks'],
 'data-section-id': 'today',
 'data-module-id': 'startpage|today|todaystripe|today.Hero|todayStripeHeadlines',
 'data-uipr': '2',
 'data-aop': 'today.hero_todaystripeheadlines',
 'data-id': '117',
 'data-m': '{"i":117,"p":25,"n":"today.Hero","t":"todayStripeHeadlines","o":4}'}

In [97]:
soup.select('.todaystripe.grid')[0].children.select('li')

AttributeError: 'list_iterator' object has no attribute 'select'

In [90]:
for xyz, zyx in enumerate(soup.select('.todaystripe.grid')[0].children):
    print(xyz)
    print(zyx)

0


1
<h2 class="hide">Latest Stories</h2>
2


3
<ul>
<li class="infopane-placeholder">
<div class="ip infopanestripe slideshow" data-aop="infopane_a_infopanestripeheadlines" data-id="118" data-m='{"i":118,"p":117,"n":"infopane_a","t":"infopaneStripeHeadlines","o":1}' data-module-id="startpage|infopane|infopanestripe|infopane_a|infopaneStripeHeadlines" data-placements="infopane-triptych-1:3;infopane-triptych-2:3;infopane-triptych-3:3;News-Stripe:1;Sports-Stripe:1;Entertainment-Stripe:1;Lifestyle-Stripe:1" data-positions="4,9,12" data-section-id="infopane" data-uipr="1">
<ul class="swipenav">
<li class="mosaic hasimage selected" data-aop="slide1&gt;mosaic " data-id="119" data-m='{"i":119,"p":118,"n":"mosaic","y":8,"o":1}' tabindex="0">
<h3>G-7 SUMMIT</h3>
<a data-id="120" data-m='{"i":120,"p":119,"n":"HeadlineItemViewModel","y":13,"l":"AAGkZmU","h":0,"v":"news","c":"newsworld","o":1,"s":1}' href="/en-us/news/world/trump-claims-g-7-unity-on-iran-after-unannounced-invite/ar-AAGkZmU?li=BBn

In [86]:
for xyz in soup.select('.todaystripe.grid')[0].descendants:
    print(xyz)



<h2 class="hide">Latest Stories</h2>
Latest Stories


<ul>
<li class="infopane-placeholder">
<div class="ip infopanestripe slideshow" data-aop="infopane_a_infopanestripeheadlines" data-id="118" data-m='{"i":118,"p":117,"n":"infopane_a","t":"infopaneStripeHeadlines","o":1}' data-module-id="startpage|infopane|infopanestripe|infopane_a|infopaneStripeHeadlines" data-placements="infopane-triptych-1:3;infopane-triptych-2:3;infopane-triptych-3:3;News-Stripe:1;Sports-Stripe:1;Entertainment-Stripe:1;Lifestyle-Stripe:1" data-positions="4,9,12" data-section-id="infopane" data-uipr="1">
<ul class="swipenav">
<li class="mosaic hasimage selected" data-aop="slide1&gt;mosaic " data-id="119" data-m='{"i":119,"p":118,"n":"mosaic","y":8,"o":1}' tabindex="0">
<h3>G-7 SUMMIT</h3>
<a data-id="120" data-m='{"i":120,"p":119,"n":"HeadlineItemViewModel","y":13,"l":"AAGkZmU","h":0,"v":"news","c":"newsworld","o":1,"s":1}' href="/en-us/news/world/trump-claims-g-7-unity-on-iran-after-unannounced-invite/ar-AAGkZmU

<span class="radio-icon"><span></span></span>
<span></span>
2




<li aria-checked="false" aria-labelledby="pa-aaglx5o-en-us-campaign-enus-amp-3-label" aria-posinset="4" aria-setsize="5" role="radio">
<input data-id="233" data-m='{"i":233,"p":223,"n":"poll_answer","y":10,"o":10}' id="pa-aaglx5o-en-us-campaign-enus-amp-3" name="pa-aaglx5o-en-us-campaign-enus-amp" type="radio" value="3"/>
<label class="answer" for="pa-aaglx5o-en-us-campaign-enus-amp-3" id="pa-aaglx5o-en-us-campaign-enus-amp-3-label"><span class="radio-icon"><span></span></span>3</label>
</li>


<input data-id="233" data-m='{"i":233,"p":223,"n":"poll_answer","y":10,"o":10}' id="pa-aaglx5o-en-us-campaign-enus-amp-3" name="pa-aaglx5o-en-us-campaign-enus-amp" type="radio" value="3"/>


<label class="answer" for="pa-aaglx5o-en-us-campaign-enus-amp-3" id="pa-aaglx5o-en-us-campaign-enus-amp-3-label"><span class="radio-icon"><span></span></span>3</label>
<span class="radio-icon"><span></span></span>
<span></span>
3




<li aria-

<label class="answer" for="pa-aagbays-en-us-campaign-enus-amp-3" id="pa-aagbays-en-us-campaign-enus-amp-3-label"><span class="radio-icon"><span></span></span>Toys “R” Us</label>
<span class="radio-icon"><span></span></span>
<span></span>
Toys “R” Us






<div class="ctr"><button data-id="292" data-m='{"i":292,"p":272,"n":"poll_submit","y":11,"o":20}' disabled="disabled" id="poll-button-aagbays-en-us-campaign-enus-amp" name="vote">ANSWER</button></div>
<button data-id="292" data-m='{"i":292,"p":272,"n":"poll_submit","y":11,"o":20}' disabled="disabled" id="poll-button-aagbays-en-us-campaign-enus-amp" name="vote">ANSWER</button>
ANSWER






<div class="results">
<header>
<button aria-label="previous quiz results" class="prev" type="button">Previous</button>
<h3>QUIZ RESULTS</h3>
<button aria-label="next quiz results" class="next" type="button">Next</button>
</header>
<h4>Where a kid can be a kid</h4>
<ul>
<li>
<div class="resultsdivpadding">
<div class="value">2%</div>
<div class="answe

In [84]:
for zz in soup.select('.todaystripe.grid')[0].descendants.select('li'):
              print(zz['class'][0])

AttributeError: 'generator' object has no attribute 'select'

In [51]:
soup

<!DOCTYPE html>
<html class="hiperf" dir="ltr" lang="en-US" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#" style="font-size:62.5%">
<head data-client-settings='{"geo_country":"us","geo_subdivision":"new york","geo_zip":"10006","geo_ip":"4.16.132.0","geo_lat":"40.7064","geo_long":"-74.0155","os_region":"","apps_locale":"","base_url":"/en-us/homepage/","aid":"1fc9b86a4539446990913fc1d41fb869","sid":null,"v":"20190816_17716915","static_page":false,"empty_gif":"//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif","functionalonly_cookie_experience":false,"functional_cookies":"","functional_cookie_patterns":"","fbid":"132970837947","lvk":"homepage","vk":"homepage","cat":null,"autorefresh":true,"bingssl":false,"autorefreshsettings":{"is_market_enabled":false,"timeout":0,"idle_enabled":false,"idle_timeout":"2"},"uipr":true,"uiprsettings":{"enabled":true,"frequency_minutes":5,"banner_delay_minutes":null,"maxfresh_display":null,"minfresh_count":"5","ajaxtimeoutinseconds":"60

In [10]:
for index, item in enumerate(soup.select('.stripecontainer[aria-label]'), start = 2):

        print(item['aria-label'])


new york
news
from our partners
entertainment
sports
money
lifestyle
autos
health & fitness
food & drink
travel
causes
video
promotions


In [11]:
ib.attrs

NameError: name 'ib' is not defined

In [109]:
re.findall("\"o\":(\d+)",ib['data-m'])[0]

'2'

In [110]:
ibl

<li data-id="624" data-m='{"i":624,"p":621,"n":"HeadlineItemViewModel","y":13,"l":"AAGh0FD","h":0,"v":"foodanddrink","c":"foodnews","o":3}'>
<a href="/en-us/foodanddrink/foodnews/the-ceo-of-popeyes-parent-company-breaks-down-the-massive-success-of-its-chicken-sandwich/ar-AAGh0FD?li=BBnb7Kw">
<img alt="" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img?h=75&amp;w=100&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;x=902&amp;y=1027"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif" title="The CEO of Popeyes' parent company breaks down the massive success of its chicke... - Irene Jiang / Business Insider"/> <div>
<h3 aria-label="popeyes' parent company ceo on chicken sandwich success-article">Popeyes' parent company CEO on chicken sandwich success</h3>
<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp

In [111]:
ibl.attrs

{'data-id': '624',
 'data-m': '{"i":624,"p":621,"n":"HeadlineItemViewModel","y":13,"l":"AAGh0FD","h":0,"v":"foodanddrink","c":"foodnews","o":3}'}

In [16]:
item_block_link.attrs

{'data-id': '673',
 'data-m': '{"i":673,"p":670,"n":"HeadlineItemViewModel","y":13,"l":"BBTBwqL","h":0,"v":"foodanddrink","c":"foodnews","o":3}',
 'class': ['media', 'photo']}

In [23]:
x

<li data-id="672" data-m='{"i":672,"p":670,"n":"HeadlineItemViewModel","y":13,"l":"AAGh0FD","h":0,"v":"foodanddrink","c":"foodnews","o":2}'>
<a href="/en-us/foodanddrink/foodnews/the-ceo-of-popeyes-parent-company-breaks-down-the-massive-success-of-its-chicken-sandwich/ar-AAGh0FD?li=BBnb7Kw">
<img alt="" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img?h=75&amp;w=100&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;x=902&amp;y=1027"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif" title="The CEO of Popeyes' parent company breaks down the massive success of its chicke... - Irene Jiang / Business Insider"/> <div>
<h3 aria-label="popeyes' parent company ceo on chicken sandwich success-article">Popeyes' parent company CEO on chicken sandwich success</h3>
<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp

In [44]:
re.findall('\/\/(.+)\"', x.a.img['data-src'])[0]

'static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img?h=75&w=100&m=6&q=60&u=t&o=t&l=f&x=902&y=1027'

In [45]:
x.a.img['data-src']

'{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img?h=75&w=100&m=6&q=60&u=t&o=t&l=f&x=902&y=1027"}'

In [31]:
x.a['href']

'/en-us/foodanddrink/foodnews/the-ceo-of-popeyes-parent-company-breaks-down-the-massive-success-of-its-chicken-sandwich/ar-AAGh0FD?li=BBnb7Kw'

In [46]:
df = []
for index, item in enumerate(soup.select('.stripecontainer[aria-label]'), start = 2):
    if item.select('img[alt="Business Insider Logo"]'):
#         print(item)
        print(index)
        river_position = index
        river_name = item['aria-label']
        a = item

        for item_block in item.find_all('ul'):
            if item_block.select('img[alt="Business Insider Logo"]'):
                ib = item_block
                item_block_type = item_block['data-aop']
                item_block_position = re.findall("\"o\":(\d+)",item_block['data-m'])[0]
#                 print(item_block)
                for item_block_link in item_block.select('li'):
                    if item_block_link.select('img[alt="Business Insider Logo"]'):
                        ibl = item_block_link
                        x = item_block_link
                        print(item_block_link)
                        print(len(item_block_link))
                        item_block_link_position = re.findall("\"o\":(\d+)",item_block_link['data-m'])[0]
                        article_tile = item_block_link.select('h3[aria-label]')[0]['aria-label']
                        article_link = item_block_link.a['href']
                        article_img = re.findall(r'\/\/(.*)\?',item_block_link.select('img')[0]['data-src'])[0]
                        article_img_small = re.findall('\/\/(.+)\"', item_block_link.a.img['data-src'])[0]
#                         artile_img = re.findall(r'\/\/(.*)\?',item_block_link.select('img')[0]['data-src'])[0]
                        df.append([river_position, 
                                   river_name,
                                   item_block_position,
                                   item_block_type,
                                   item_block_link_position,
                                   article_tile,
                                   article_link,
                                   article_img,
                                   article_img_small])
        
        
    else:
        print('***')
pd.DataFrame(df, columns = ['river_position',
                           'river_name',
                           'item_block_position',
                           'item_block_type',
                           'item_block_link_position',
                           'article_title',
                           'article_link',
                           'article_img',
                           'article_img_small'])

***
***
***
***
***
***
***
***
***
11
<li data-id="672" data-m='{"i":672,"p":670,"n":"HeadlineItemViewModel","y":13,"l":"AAGh0FD","h":0,"v":"foodanddrink","c":"foodnews","o":2}'>
<a href="/en-us/foodanddrink/foodnews/the-ceo-of-popeyes-parent-company-breaks-down-the-massive-success-of-its-chicken-sandwich/ar-AAGh0FD?li=BBnb7Kw">
<img alt="" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img?h=75&amp;w=100&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;x=902&amp;y=1027"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif" title="The CEO of Popeyes' parent company breaks down the massive success of its chicke... - Irene Jiang / Business Insider"/> <div>
<h3 aria-label="popeyes' parent company ceo on chicken sandwich success-article">Popeyes' parent company CEO on chicken sandwich success</h3>
<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityi

,river_position,river_name,item_block_position,item_block_type,item_block_link_position,article_title,article_link,article_img,article_img_small
0,11,food & drink,2,tertiary,2,popeyes' parent company ceo on chicken sandwic...,/en-us/foodanddrink/foodnews/the-ceo-of-popeye...,static-global-s-msn-com.akamaized.net/img-resi...,static-global-s-msn-com.akamaized.net/img-resi...


In [77]:
x

<li data-id="624" data-m='{"i":624,"p":621,"n":"HeadlineItemViewModel","y":13,"l":"AAGh0FD","h":0,"v":"foodanddrink","c":"foodnews","o":3}'>
<a href="/en-us/foodanddrink/foodnews/the-ceo-of-popeyes-parent-company-breaks-down-the-massive-success-of-its-chicken-sandwich/ar-AAGh0FD?li=BBnb7Kw">
<img alt="" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img?h=75&amp;w=100&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;x=902&amp;y=1027"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif" title="The CEO of Popeyes' parent company breaks down the massive success of its chicke... - Irene Jiang / Business Insider"/> <div>
<h3 aria-label="popeyes' parent company ceo on chicken sandwich success-article">Popeyes' parent company CEO on chicken sandwich success</h3>
<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp

In [82]:
x.select('h3[aria-label]')[0]['aria-label']

"popeyes' parent company ceo on chicken sandwich success-article"

In [97]:
re.findall(r'\/\/(.*)\?',x.select('img[alt=""]')[0]['data-src'])[0]

'static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img'

In [92]:
x.select('img[alt=""]')[0]['data-src']

'{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAGgYGw.img?h=75&w=100&m=6&q=60&u=t&o=t&l=f&x=902&y=1027"}'

In [57]:
a.select('.stripecontainer[aria-label]')

[]

In [63]:
a.find_all('ul')

[<ul>
 <li data-id="604" data-m='{"i":604,"p":603,"n":"foodanddrink","y":4,"o":1}'>
 <h2><a data-id="605" data-m='{"i":605,"p":603,"n":"foodanddrink","y":4,"o":2}' href="/en-us/foodanddrink">Food &amp; Drink</a></h2>
 </li>
 <li class="hide12" data-id="606" data-m='{"i":606,"p":603,"n":"Food News","y":4,"o":3}'><a href="/en-us/foodanddrink/restaurantsandnews">FOOD NEWS</a></li>
 <li class="hide12" data-id="607" data-m='{"i":607,"p":603,"n":"Restaurants","y":4,"o":4}'><a href="/en-us/foodanddrink/restaurantsandnews/casual">RESTAURANTS</a></li>
 <li class="hide12" data-id="608" data-m='{"i":608,"p":603,"n":"Recipes","y":4,"o":5}'><a href="/en-us/foodanddrink/cooking/recipes">RECIPES</a></li>
 <li class="hide12" data-id="609" data-m='{"i":609,"p":603,"n":"Quick \u0026 Easy","y":4,"o":6}'><a href="/en-us/foodanddrink/cooking/quickandeasy">QUICK &amp; EASY</a></li>
 <li class="hide12" data-id="610" data-m='{"i":610,"p":603,"n":"Tips \u0026 Tricks","y":4,"o":7}'><a href="/en-us/foodanddrink/

In [62]:
a

<div aria-label="food &amp; drink" class="stripecontainer" data-section-id="stripe.foodanddrink" role="region">
<div class="fooddrink stripeouter" data-aop="stripe.foodanddrink_stripe" data-id="601" data-m='{"i":601,"p":25,"n":"stripe.foodanddrink","t":"stripe","o":14}'>
<button class="leftarrow" data-id="602" data-m='{"i":602,"p":601,"n":"leftarrow","y":12,"o":1}' tabindex="-1" title="previous"></button>
<div class="stripe full-width">
<div class="stripenav" data-aop="stripe.foodanddrink.navigation_stripenavigation" data-id="603" data-m='{"i":603,"p":601,"n":"stripe.foodanddrink.navigation","t":"stripeNavigation","o":2}'>
<ul>
<li data-id="604" data-m='{"i":604,"p":603,"n":"foodanddrink","y":4,"o":1}'>
<h2><a data-id="605" data-m='{"i":605,"p":603,"n":"foodanddrink","y":4,"o":2}' href="/en-us/foodanddrink">Food &amp; Drink</a></h2>
</li>
<li class="hide12" data-id="606" data-m='{"i":606,"p":603,"n":"Food News","y":4,"o":3}'><a href="/en-us/foodanddrink/restaurantsandnews">FOOD NEWS</a

In [59]:
a.attrs

{'class': ['stripecontainer'],
 'role': 'region',
 'aria-label': 'food & drink',
 'data-section-id': 'stripe.foodanddrink'}

In [61]:
a.select('img[alt="Business Insider Logo"]')

[<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=png"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif"/>]

In [38]:
df

[[11, 'food & drink'], {11, 'food & drink'}]

In [39]:
pd.DataFrame(df)

,0,1
0,11,food & drink
1,food & drink,11


In [44]:
soup.select('.stripecontainer[aria-label]')[9].select('img[alt="Business Insider Logo"]')

[<img alt="Business Insider Logo" data-src='{"default":"//static-global-s-msn-com.akamaized.net/img-resizer/tenant/amp/entityid/AAywSGf.img?h=16&amp;w=16&amp;m=6&amp;q=60&amp;u=t&amp;o=t&amp;l=f&amp;f=png"}' src="//static-global-s-msn-com.akamaized.net/hp-eus/sc/9b/e151e5.gif"/>]